# Function detect_simpsons_paradox Computational Complexity Analysis
The number of records: $n$  
The total number of attributes: $a$  
The number of group-by attributes(discrete vriable): $c$  
The levels for each group-by attribute: $k_i$  
The number of continouse attributes: $d$   

### Separate the attributes by types
$O(a)$:    groupbyAttrs = latent_df.select_dtypes(include=['object','int64'])<br/>
$O(a)$:    continuousAttrs = latent_df.select_dtypes(include=['float64'])<br/>
$O(1)$:    continuousAttrs_labels = list(continuousAttrs)<br/>
$O(1)$:    groupbyAttrs_labels = list(groupbyAttrs)<br/>

#### Total: $O(a)$

### Compute correlation for all of the data
$O(d*d*n)$:    all_corr = latent_df[continuousAttrs_labels].corr()  
$O(d*d)$:      all_corr_df = upper_triangle_df(all_corr)  
$O(d*d)$:      all_corr_element = all_corr_df['value'].values  

#### Total:  $O(d*d*n)$

### Compute subgroup correlation matrix, then compare to the all data correlation matrix

* $O(c)$:for groupbyAttr in groupbyAttrs_labels:  
    * $O(d*d*n)$:    grouped_df_corr = latent_df.groupby(groupbyAttr)[continuousAttrs_labels].corr()  
    * $O(1)$:        groupby_value=grouped_df_corr.index.get_level_values(groupbyAttr).unique()  
  
    * $O(k_i)$:  for subgroup in groupby_value:  
        * $O(1)$:        subgroup_corr = grouped_df_corr.loc[subgroup]  
        * $O(d*d)$:      subgroup_corr_elements = upper_triangle_element(subgroup_corr)  
        * $O(d*d)$:      index_list = [i for i, (a,b) in enumerate(zip(all_corr_element, subgroup_corr_elements)) if isReverse(a, b)]  
        * $O(d*d)$:      reverse_list = [j for i, j in zip(all_corr_element, subgroup_corr_elements) if isReverse(i, j)]  
        
        * Ignore!!!:     if reverse_list:  
            * Ignore!!!:     all_corr_info = [all_corr_df.loc[i].values for i in index_list]
            * Ignore!!!:     temp_df = pd.DataFrame(data=all_corr_info,columns=['allCorr','attr1','attr2'])
            * Ignore!!!:     temp_df.attr1 = temp_df.attr1.astype(int)  
            * Ignore!!!:     temp_df.attr2 = temp_df.attr2.astype(int)  
            * Ignore!!!:     temp_df["reverseCorr"] = reverse_list  
            * Ignore!!!:     len_list = len(reverse_list)  
            * Ignore!!!:     temp_df['groupbyAttr'] = [groupbyAttr for i in range(len_list)]  
            * Ignore!!!:     temp_df['subgroup'] = [subgroup for i in range(len_list)]  
            * Ignore!!!:     result_df = result_df.append(temp_df, ignore_index=True)  
                            
#### Total: $O(c*d*d*n + c*k_i*d*d)$ ==>  $O(c*d*d*n)$

# Computational Complexity for all: $O(c*d*d*n)$